# Todo el Proceso de ML con AutoGluon

Aqui utilizaremos la libreria [AutoGluon](https://auto.gluon.ai/) para realizar todo el proceso de ML de forma automatica. AutoGluon es un framework de AutoML desarrollado por Amazon que automatiza el preprocesamiento de datos, el entrenamiento de multiples modelos, la optimizacion de hiperparametros y la creacion de ensambles para obtener las mejores predicciones posibles con minimo codigo.

Comencemos instalando las librerias necesarias.

In [ ]:
!pip install autogluon -U -q

Importamos AutoGluon y vemos su version

In [ ]:
import autogluon

Importamos las dos clases principales que usaremos:
- **TabularDataset**: para cargar datos tabulares (es una subclase de pandas DataFrame)
- **TabularPredictor**: para entrenar modelos y hacer predicciones

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

Cargaremos ahora los datos sin procesar de nuestro ejemplo de diabetes que vimos durante todo el curso.

In [ ]:
import pandas as pd
df_ana_train = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_train_analisis.csv')
df_inf_train = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_train_info.csv')
df_train = pd.merge(df_ana_train, df_inf_train, on="id")
df_train = df_train.drop("id", axis=1)
df_train.head()

Convertimos nuestro DataFrame de pandas a un TabularDataset de AutoGluon. Esto es opcional ya que TabularPredictor tambien acepta DataFrames de pandas directamente.

In [ ]:
train_data = TabularDataset(df_train)
train_data.head()

## Entrenamiento

Ahora entrenamos nuestro modelo con AutoGluon. Solo necesitamos especificar:
- **label**: la columna objetivo que queremos predecir
- **eval_metric**: la metrica que usaremos para evaluar los modelos

AutoGluon automaticamente:
1. Detecta si es un problema de clasificacion o regresion
2. Realiza el preprocesamiento de los datos
3. Entrena multiples modelos (LightGBM, CatBoost, XGBoost, Random Forest, Redes Neuronales, etc.)
4. Crea un ensamble de los mejores modelos

Usamos `time_limit` para limitar el tiempo de entrenamiento en segundos. Un tiempo mayor generalmente produce mejores resultados.

In [ ]:
predictor = TabularPredictor(
    label='diabetes',
    eval_metric='accuracy'
).fit(
    train_data,
    time_limit=120
)

## Analisis de los modelos entrenados

Podemos ver un resumen de todos los modelos que AutoGluon ha entrenado, ordenados por su rendimiento en el conjunto de validacion. Esto es similar al `compare_models()` de PyCaret.

In [ ]:
predictor.leaderboard(silent=True)

Podemos ver un resumen general del entrenamiento

In [ ]:
predictor.fit_summary()

Veamos la importancia de cada columna (feature importance) para el mejor modelo. Esto nos indica que variables tienen mayor impacto en la prediccion.

In [ ]:
predictor.feature_importance(train_data)

## Predicciones sobre datos de prueba

Cargamos el dataset de prueba

In [ ]:
df_ana_test = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_test_analisis.csv')
df_inf_test = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_test_info.csv')
df_test = pd.merge(df_ana_test, df_inf_test, on="id")
df_test = df_test.drop("id", axis=1)
test_data = TabularDataset(df_test)
test_data.head()

Hacemos predicciones sobre nuevos valores

In [ ]:
y_pred = predictor.predict(test_data.drop(columns=['diabetes']))
y_pred.head()

Tambien podemos obtener las probabilidades de cada clase

In [ ]:
y_pred_proba = predictor.predict_proba(test_data.drop(columns=['diabetes']))
y_pred_proba.head()

## Evaluacion

Evaluamos el rendimiento del predictor sobre los datos de prueba. AutoGluon calcula automaticamente las metricas relevantes.

In [ ]:
predictor.evaluate(test_data)

Tambien podemos ver el leaderboard con los resultados sobre los datos de prueba para comparar como funcionan los distintos modelos

In [ ]:
predictor.leaderboard(test_data, silent=True)

## Guardar y cargar el modelo

Los modelos se guardan automaticamente y luego podemos cargarlos utilizando la funcion load. Para guardarlos manualmente podemos utilizar la funcion save.

```python
loaded_predictor = TabularPredictor.load('path a mi modelo de autogluon')
```

## Presets: controlando la calidad y velocidad del entrenamiento

AutoGluon ofrece distintos **presets** que permiten balancear entre calidad de prediccion y tiempo de entrenamiento:

| Preset | Descripcion |
|--------|-------------|
| `medium` | Entrenamiento rapido, ideal para prototipos |
| `good` | Buena precision con inferencia muy rapida |
| `high` | Alta precision con buena velocidad de inferencia |
| `best` | Maximiza la precision, recomendado para competencias |

Ejemplo de uso con preset:

In [ ]:
predictor_best = TabularPredictor(
    label='diabetes',
    eval_metric='accuracy'
).fit(
    train_data,
    presets='best',
    time_limit=120
)

predictor_best.leaderboard(test_data, silent=True)

## Referencias:

- https://auto.gluon.ai/stable/tutorials/tabular/tabular-quick-start.html
- https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html
- https://github.com/autogluon/autogluon

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/datascience/datascience.html)